In [1]:
!pip install gdown -q

!gdown --id 14Zt1EXmBWimBKBMOjQuwrlBY475d0Yze -O /content/IF25-4041-dataset.zip -q
!unzip -q /content/IF25-4041-dataset.zip -d /content/dataset

# Instalasi Dependensi
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q
!pip install timm albumentations torchinfo scikit-learn pandas numpy matplotlib seaborn -q


/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(


In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Albumentations untuk augmentasi
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Scikit-learn
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Vision models
import timm
from torchinfo import summary

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device yang digunakan: {device}")

# Definisi kelas (urutan ini akan jadi mapping label → index)
classes = ['bakso', 'gado_gado', 'nasi_goreng', 'rendang', 'soto_ayam']
num_classes = len(classes)
print(f"Jumlah kelas: {num_classes}")
print(f"Kelas: {classes}")

In [ ]:
class FoodDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, is_test=False, classes=None):
        """
        csv_file : path ke file CSV (harus ada kolom 'filename' dan 'label' untuk train/val)
        img_dir  : folder tempat gambar disimpan
        """
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test

        # mapping label -> index (pakai urutan 'classes' yang kamu definisikan di atas)
        if not is_test:
            assert classes is not None, "Parameter 'classes' wajib diisi untuk mode train/val."
            self.label_to_idx = {label: idx for idx, label in enumerate(classes)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx]['filename']
        img_path = os.path.join(self.img_dir, img_name)

        # Load image (RGB)
        image = Image.open(img_path).convert('RGB')

        # Albumentations butuh array numpy
        image = np.array(image)

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']  # sudah jadi tensor CxHxW

        if self.is_test:
            # mode test: tidak ada label
            return image, img_name
        else:
            label = self.data.iloc[idx]['label']
            label_idx = self.label_to_idx[label]
            return image, label_idx
